# jaccard 유사도를 활용한 Document 비교

## Library Import

In [1]:
import numpy as np
import pandas as pd

## data 불러오기


In [145]:
df=pd.read_excel('./jaccard_source.xlsx') # data 불러오기

## data 확인하기

In [146]:
print(df.shape)
# df

(18, 2)


# DataFrame에서 Jaccard similarity 구하기

## Concatanate by Groupby

In [229]:
df_agg=df.groupby(["문서명"])["Data"].apply(lambda x: ",".join(x)).reset_index()
r=df_agg.shape[0]
df_agg_repeat1 = pd.DataFrame(np.repeat(df_agg.values, r, axis=0), columns=df.columns)
df_agg_repeat1['문서명2']=np.tile(df_agg['문서명'],r) # np.tile이용 (순서 그대로 반복)
df_agg_repeat1['Data2']=np.tile(df_agg['Data'],r)
df_agg_repeat1


,문서명,Data,문서명2,Data2
0,A,"이름,나이,주소,별명,연락처,이메일",A,"이름,나이,주소,별명,연락처,이메일"
1,A,"이름,나이,주소,별명,연락처,이메일",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
2,A,"이름,나이,주소,별명,연락처,이메일",C,"이메일,연락처,SNS,우편번호,저축"
3,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",A,"이름,나이,주소,별명,연락처,이메일"
4,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
5,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",C,"이메일,연락처,SNS,우편번호,저축"
6,C,"이메일,연락처,SNS,우편번호,저축",A,"이름,나이,주소,별명,연락처,이메일"
7,C,"이메일,연락처,SNS,우편번호,저축",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
8,C,"이메일,연락처,SNS,우편번호,저축",C,"이메일,연락처,SNS,우편번호,저축"


In [230]:
# np.tile(df_agg['문서명'],r) #array순서대로 반영하기
# np.tile(df_agg['Data'],r)

## Apply 함수에 적용할 사용자 정의함수 만들기

In [231]:
# 교집합 연산
def str_compare_intersects(str1,str2):
    s1=set(str1.split(","))
    s2=set(str2.split(","))
    intersect=list(s1&s2)
    str_intersect=",".join(intersect)
    
    return str_intersect
# 교집합 원소 개수
def str_compare_intersects_counts(str1,str2):
    s1=set(str1.split(","))
    s2=set(str2.split(","))
    intersect=list(s1&s2)
    num_intersect=len(intersect)
    
    return num_intersect
# 합집합 연산
def str_compare_union(str1,str2):
    s1=set(str1.split(","))
    s2=set(str2.split(","))
    union=list(s1|s2)
    str_union=",".join(union)
    
    return str_union
# 합집합 원소 개수
def str_compare_union_counts(str1,str2):
    s1=set(str1.split(","))
    s2=set(str2.split(","))
    union=list(s1|s2)
    num_union=len(union)
    
    return num_union

    

## DataFrame Deepcopy

In [232]:
df_agg_repeat2=df_agg_repeat1.copy()
df_agg_repeat2



,문서명,Data,문서명2,Data2
0,A,"이름,나이,주소,별명,연락처,이메일",A,"이름,나이,주소,별명,연락처,이메일"
1,A,"이름,나이,주소,별명,연락처,이메일",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
2,A,"이름,나이,주소,별명,연락처,이메일",C,"이메일,연락처,SNS,우편번호,저축"
3,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",A,"이름,나이,주소,별명,연락처,이메일"
4,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
5,B,"연락처,이메일,SNS,우편번호,별명,주소,투자",C,"이메일,연락처,SNS,우편번호,저축"
6,C,"이메일,연락처,SNS,우편번호,저축",A,"이름,나이,주소,별명,연락처,이메일"
7,C,"이메일,연락처,SNS,우편번호,저축",B,"연락처,이메일,SNS,우편번호,별명,주소,투자"
8,C,"이메일,연락처,SNS,우편번호,저축",C,"이메일,연락처,SNS,우편번호,저축"


In [233]:
# str_compare_intersects(df_agg_repeat2["Data"][2] ,df_agg_repeat2["Data2"][2])

## apply + lambda 활용하기

### 소수점 자리수 지정

In [235]:
pd.set_option('display.precision', 6)

### Dataframe에 apply + lambda 적용 : 사용자 정의 함수 활용

In [236]:
# Dataframe에 lambda 적용 : axix=1 옵션 지정 - Columns끼리 연산

#1. 중복된 문서 비교 자료 제거위한 Column 추가
df_agg_repeat2["duplicates"]=df_agg_repeat2["문서명"]+","+df_agg_repeat2["문서명2"]
df_agg_repeat2["duplicates"]=df_agg_repeat2["duplicates"].apply(lambda x : ",".join(sorted(list(x.split(",")))))
#2. Data간 합집합/교집합 연산하기
df_agg_repeat2["intersect_counts"]=df_agg_repeat2.apply(lambda x : str_compare_intersects_counts(x["Data"],x["Data2"]),axis=1)
df_agg_repeat2["union_counts"]=df_agg_repeat2.apply(lambda x : str_compare_union_counts(x["Data"],x["Data2"]),axis=1)
df_agg_repeat2["similarity"]=df_agg_repeat2["intersect_counts"]/df_agg_repeat2["union_counts"]
df_agg_repeat2["union"]=df_agg_repeat2.apply(lambda x : str_compare_union(x["Data"],x["Data2"]),axis=1)
df_agg_repeat2["intersects"]=df_agg_repeat2.apply(lambda x : str_compare_intersects(x["Data"],x["Data2"]),axis=1)
df_agg_repeat2.head(2)


,문서명,Data,문서명2,Data2,duplicates,intersect_counts,union_counts,similarity,union,intersects
0,A,"이름,나이,주소,별명,연락처,이메일",A,"이름,나이,주소,별명,연락처,이메일","A,A",6,6,1.000000,"별명,연락처,이름,주소,이메일,나이","별명,연락처,이름,주소,이메일,나이"
1,A,"이름,나이,주소,별명,연락처,이메일",B,"연락처,이메일,SNS,우편번호,별명,주소,투자","A,B",4,9,0.444444,"우편번호,투자,별명,연락처,SNS,이름,주소,이메일,나이","이메일,주소,별명,연락처"


## 문자열 수정 : 쉼표 -> 줄바꿈

In [237]:
# 줄바꿈 : 엑셀에서 자동 줄바꿈 옵션 지정해서 줄 바꾼 형태로 바뀜

In [238]:
df_agg_repeat2["Data"]=df_agg_repeat2["Data"].apply(lambda x : x.replace(",","\n"))
df_agg_repeat2["Data2"]=df_agg_repeat2["Data2"].apply(lambda x : x.replace(",","\n"))
df_agg_repeat2["union"]=df_agg_repeat2["union"].apply(lambda x : x.replace(",","\n"))
df_agg_repeat2["intersects"]=df_agg_repeat2["intersects"].apply(lambda x : x.replace(",","\n"))
df_agg_repeat2

,문서명,Data,문서명2,Data2,duplicates,intersect_counts,union_counts,similarity,union,intersects
0,A,이름\n나이\n주소\n별명\n연락처\n이메일,A,이름\n나이\n주소\n별명\n연락처\n이메일,"A,A",6,6,1.000000,별명\n연락처\n이름\n주소\n이메일\n나이,별명\n연락처\n이름\n주소\n이메일\n나이
1,A,이름\n나이\n주소\n별명\n연락처\n이메일,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,"A,B",4,9,0.444444,우편번호\n투자\n별명\n연락처\nSNS\n이름\n주소\n이메일\n나이,이메일\n주소\n별명\n연락처
2,A,이름\n나이\n주소\n별명\n연락처\n이메일,C,이메일\n연락처\nSNS\n우편번호\n저축,"A,C",2,9,0.222222,저축\n우편번호\n별명\n연락처\nSNS\n이름\n주소\n이메일\n나이,이메일\n연락처
3,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,A,이름\n나이\n주소\n별명\n연락처\n이메일,"A,B",4,9,0.444444,우편번호\n투자\n연락처\n별명\nSNS\n이름\n주소\n이메일\n나이,이메일\n주소\n별명\n연락처
4,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,"B,B",7,7,1.000000,우편번호\n투자\n연락처\n별명\nSNS\n주소\n이메일,이메일\n투자\n우편번호\n연락처\n별명\n주소\nSNS
5,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,C,이메일\n연락처\nSNS\n우편번호\n저축,"B,C",4,8,0.500000,저축\n우편번호\n투자\n연락처\n별명\nSNS\n주소\n이메일,SNS\n이메일\n우편번호\n연락처
6,C,이메일\n연락처\nSNS\n우편번호\n저축,A,이름\n나이\n주소\n별명\n연락처\n이메일,"A,C",2,9,0.222222,저축\n우편번호\n연락처\n별명\nSNS\n이름\n주소\n이메일\n나이,이메일\n연락처
7,C,이메일\n연락처\nSNS\n우편번호\n저축,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,"B,C",4,8,0.500000,저축\n우편번호\n투자\n연락처\n별명\nSNS\n주소\n이메일,SNS\n이메일\n우편번호\n연락처
8,C,이메일\n연락처\nSNS\n우편번호\n저축,C,이메일\n연락처\nSNS\n우편번호\n저축,"C,C",5,5,1.000000,저축\n우편번호\n연락처\nSNS\n이메일,저축\n이메일\n우편번호\n연락처\nSNS


## 비교 결과중 중복된 결과 제거하기

In [239]:
df_agg_result_for_listup=df_agg_repeat2.drop_duplicates(["duplicates"],keep="first") # 중복행 삭제, 중복 값중 첫번째 값만 남김
df_agg_result_for_listup.reset_index(drop=True,inplace=True) # 기존의 인덱스열 drop
df_agg_result_for_listup.head(2)

,문서명,Data,문서명2,Data2,duplicates,intersect_counts,union_counts,similarity,union,intersects
0,A,이름\n나이\n주소\n별명\n연락처\n이메일,A,이름\n나이\n주소\n별명\n연락처\n이메일,"A,A",6,6,1.000000,별명\n연락처\n이름\n주소\n이메일\n나이,별명\n연락처\n이름\n주소\n이메일\n나이
1,A,이름\n나이\n주소\n별명\n연락처\n이메일,B,연락처\n이메일\nSNS\n우편번호\n별명\n주소\n투자,"A,B",4,9,0.444444,우편번호\n투자\n별명\n연락처\nSNS\n이름\n주소\n이메일\n나이,이메일\n주소\n별명\n연락처


## CrossTable 집계하기

In [240]:
pd.set_option('display.precision', 4)

In [241]:
df_result=df_agg_repeat2[["문서명","문서명2","similarity"]]
# Matrix형태로 Data저장하기
df_matrix=pd.crosstab(df_result["문서명"],df_result["문서명2"],values=df_result["similarity"],aggfunc=np.sum)
df_matrix

문서명2,A,B,C
문서명,,,
A,1.0000,0.4444,0.2222
B,0.4444,1.0000,0.5000
C,0.2222,0.5000,1.0000


## 결과 저장하기

In [242]:
writer=pd.ExcelWriter("./jacc_result_r2.xlsx")

In [243]:
df_agg_result_for_listup.to_excel(writer,sheet_name="sheet1")
df_matrix.to_excel(writer,sheet_name="sheet2")
writer.save()